# Building a LinkedIn Job Search Engine Using Python Web Scraping

This project is inspired by https://maoviola.medium.com/a-complete-guide-to-web-scraping-linkedin-job-postings-ad290fcaa97f

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import time
import pandas as pd

In [7]:
# Define browser and action setup
PATH = "C:\\Users\\JUWITA\\Downloads\\chromedriver_win32\\chromedriver.exe"
driver = webdriver.Chrome(PATH)

# Define URL
linkedin_url = "https://www.linkedin.com/jobs/search/?keywords=data%20scientist&location=Indonesia"

# Action Steps
driver.maximize_window()
driver.get(linkedin_url) # Open web page

# Determine how many jobs we want to scrape, and calculate how many time we need to scroll down
no_of_jobs = 1000
# int(driver.find_element_by_css_selector("h1>span").get_attribute("innerText"))
n_scroll = int(no_of_jobs/25)+1
print(n_scroll)
i = 1
driver.execute_script("return document.body.scrollHeight") #scroll to top
while i <= n_scroll:
    driver.execute_script("return document.body.scrollHeight")
    time.sleep(2) # wair for 2 seconds
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #scroll to the bottom of page
    time.sleep(3)
    i = i + 1
    try:
        button = driver.find_element(By.XPATH,"/html/body/div[1]/div/main/section[2]/button")
        #time.sleep(2)
        button.click()
        time.sleep(1)
        print("load more click")
    except:
        driver.execute_script("return document.body.scrollHeight")
        time.sleep(3)

print ("total jobs :")
jobs = driver.find_element(By.CLASS_NAME,"jobs-search__results-list").find_elements(By.TAG_NAME,'li') # return a list
print(len(jobs))

C:\Users\JUWITA\AppData\Local\Temp\ipykernel_11212\3104908011.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


41
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
total jobs :
999


In [8]:
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []
for job in jobs:
    job_id0 = job.get_attribute('data-entity-urn')
    job_id.append(job_id0)

    job_title0 = job.find_element(By.CSS_SELECTOR,'h3').get_attribute('innerText')
    job_title.append(job_title0)

    company_name0 = job.find_element(By.CSS_SELECTOR,'h4').get_attribute('innerText')
    company_name.append(company_name0)

    location0 = job.find_element(By.CLASS_NAME,'job-search-card__location').get_attribute('innerText')
    location.append(location0)

    date0 = job.find_element(By.CSS_SELECTOR,'div>div>time').get_attribute('datetime')
    date.append(date0)

    job_link0 = job.find_element(By.CSS_SELECTOR,'a').get_attribute('href')
    job_link.append(job_link0)

In [9]:
jd = []
seniority = []
emp_type = []
job_func = []
industries = []
for item in range(len(jobs)):
    job_func0=[]
    industries0=[]
    # clicking job to view job details
    try:
        job_click_path = f'/html/body/div[1]/div/main/section[2]/ul/li[{item+1}]/a/div[1]/img'
        job_click = job.find_element(By.XPATH,job_click_path).click()
    except:
        job_click_path = f'/html/body/div[1]/div/main/section[2]/ul/li[{item+1}]/div/a'
    
    print(job_click_path)
    job_click = job.find_element(By.XPATH,job_click_path).click()
    time.sleep(3)
    try:
        jd_path = '/html/body/div[1]/div/section/div[2]/div/section[1]/div/div/section'
        jd0 = job.find_element(By.XPATH,jd_path).get_attribute('innerText')
    except:
        jd_path = '/html/body/div[1]/div/section/div[2]/div/section[2]/div/div/section/div'
    
    jd0 = job.find_element(By.XPATH,jd_path).get_attribute('innerText')
    is_benefit = True
    try:
        assert 'Base pay range' in jd0
    except:
        is_benefit = False
        
    if is_benefit==True :
        jd_path = '/html/body/div[1]/div/section/div[2]/div/section[2]/div/div/section/div'
        jd0 = job.find_element(By.XPATH,jd_path).get_attribute('innerText')
        jd.append(jd0)
#         print(jd_path)
#         print("appended")
        jd_path2 = '/html/body/div[1]/div/section/div[2]/div/section[2]/div'
    else:
#         print(jd_path)
        jd.append(jd0)
#         print("appended")
        jd_path2 = '/html/body/div[1]/div/section/div[2]/div/section[1]/div'

    try:
        seniority_path = jd_path2 + '/ul/li[1]/span'
        seniority_class_name = 'description__job-criteria-text description__job-criteria-text--criteria'
        seniority0 = job.find_element(By.XPATH,seniority_path).get_attribute('innerText')
        #seniority0 = job.find_element(By.CLASS_NAME,seniority_class_name).get_attribute('innerText')
        seniority.append(seniority0)
    except:
        seniority.append('') #handling if seniority is not available
    
    try:
        emp_type_path = jd_path2 + '/ul/li[2]/span'
        emp_type_class_name = ''
        emp_type0 = job.find_element(By.XPATH,emp_type_path).get_attribute('innerText')
        #emp_type0 = job.find_element(By.CLASS_NAME,emp_type_class_name).get_attribute('innerText')
        emp_type.append(emp_type0)
    except:
        emp_type.append('') #handling if employment type is not available
    
    try:
        job_func_path = jd_path2 + '/ul/li[3]/span'
        job_func_class_name = ''
        job_func0 = job.find_element(By.XPATH,job_func_path).get_attribute('innerText')
        #job_func0 = job.find_element(By.CLASS_NAME,job_func_class_name).get_attribute('innerText')
        job_func.append(job_func0)
    except:
        job_func.append('') #handling if job function is not available
    
    try:
        industries_path = jd_path2 + '/ul/li[4]/span'
        industries_class_name = ''
        industries0 = job.find_element(By.XPATH,industries_path).get_attribute('innerText')
        #emp_type0 = job.find_element(By.CLASS_NAME,industries_class_name).get_attribute('innerText')
        industries.append(industries0)
    except:
        industries.append('')

/html/body/div[1]/div/main/section[2]/ul/li[1]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[2]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[3]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[4]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[5]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[6]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[7]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[8]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[9]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[10]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[11]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[12]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[13]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[14]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[15]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[16]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[17]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[18]/div/a
/html/body/div[1]/div/main/section[2]

/html/body/div[1]/div/main/section[2]/ul/li[151]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[152]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[153]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[154]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[155]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[156]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[157]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[158]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[159]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[160]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[161]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[162]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[163]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[164]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[165]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[166]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[167]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[168]/div/a
/html/body

/html/body/div[1]/div/main/section[2]/ul/li[300]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[301]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[302]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[303]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[304]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[305]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[306]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[307]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[308]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[309]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[310]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[311]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[312]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[313]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[314]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[315]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[316]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[317]/div/a
/html/body

/html/body/div[1]/div/main/section[2]/ul/li[449]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[450]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[451]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[452]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[453]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[454]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[455]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[456]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[457]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[458]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[459]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[460]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[461]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[462]/a/div[1]/img
/html/body/div[1]/div/main/section[2]/ul/li[463]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[464]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[465]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[466]/div/a
/ht

/html/body/div[1]/div/main/section[2]/ul/li[597]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[598]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[599]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[600]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[601]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[602]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[603]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[604]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[605]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[606]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[607]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[608]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[609]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[610]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[611]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[612]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[613]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[614]/div/a
/html/body

/html/body/div[1]/div/main/section[2]/ul/li[746]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[747]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[748]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[749]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[750]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[751]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[752]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[753]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[754]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[755]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[756]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[757]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[758]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[759]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[760]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[761]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[762]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[763]/div/a
/html/body

/html/body/div[1]/div/main/section[2]/ul/li[895]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[896]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[897]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[898]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[899]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[900]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[901]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[902]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[903]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[904]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[905]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[906]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[907]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[908]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[909]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[910]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[911]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[912]/a/div[1]/img
/ht

In [10]:
job_data = pd.DataFrame({'ID': job_id,
                        'Date': date,
                        'Company': company_name,
                        'Title': job_title,
                        'Location': location,
                        'Description' : jd,
                        'Level': seniority,
                        'Type': emp_type,
                        'Function': job_func,
                        'Industry': industries
                        })

In [11]:
job_data.head(10)

,ID,Date,Company,Title,Location,Description,Level,Type,Function,Industry
0,None,2022-12-21,Telkom Indonesia,Data Science,"Jakarta, Indonesia",Telkom Indonesia is looking for professional t...,Mid-Senior level,Contract,"Analyst, Science, and Engineering",IT Services and IT Consulting and Software Dev...
1,None,2022-12-06,PT Smartfren Telecom Tbk,Data Scientist,"Jakarta, Jakarta, Indonesia",Work with stakeholders throughout the organiza...,Entry level,Full-time,Engineering and Information Technology,Telecommunications
2,None,2022-11-30,Indodana,Data Scientist,"Jakarta, Jakarta, Indonesia",Job Description\n\n\n\n\nAnalyze data to bring...,Associate,Full-time,Engineering and Information Technology,Financial Services
3,None,2022-12-13,PT. XL Axiata Tbk,Data Scientist,"Jakarta, Indonesia",Job Purpose: Discover information hidden in va...,Associate,Full-time,Analyst and Information Technology,Telecommunications
4,None,2022-12-19,Amartha,Data Scientist,"South Jakarta City, Jakarta, Indonesia",About The Role\n\nAmartha is one of the bigges...,Not Applicable,Full-time,Engineering and Information Technology,"Technology, Information and Internet"
5,None,2022-11-21,Moladin,Machine Learning Engineer,"South Jakarta City, Jakarta, Indonesia",Design and implement cloud solutions and setup...,Not Applicable,Full-time,Engineering,"Technology, Information and Internet"
6,None,2022-12-14,Kargo Technologies,Data Scientist,"Jakarta, Jakarta, Indonesia",About The Challenges\n\nWe believe data scienc...,Not Applicable,Full-time,Engineering and Information Technology,"Technology, Information and Internet"
7,None,2022-12-09,Allianz Indonesia,Data Scientist,"Jakarta, Jakarta, Indonesia",19915 | IT & Tech Engineering | Professional |...,Not Applicable,Full-time,Engineering and Information Technology,Insurance and Financial Services
8,None,2022-10-12,ilmuOne Data,Jr Data Scientist,"Jakarta, Indonesia",We provide C-suite executives and start-up Fou...,Entry level,Full-time,Engineering and Information Technology,Software Development
9,None,2022-12-23,Koltiva,Data Scientist,"Jakarta, Jakarta, Indonesia",Job Description\n\nData Analysis takes the cen...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting


In [12]:
#change output file path to your desired path. Don't forget to put the filename and filetype also
output_file_path = 'C:\\Users\\JUWITA\\Downloads\\DataScience2.xlsx'
job_data.to_excel(output_file_path, index = False)